In [10]:
import random
import numpy
from itertools import combinations
# from bracket import bracket
from tkinter import *
from tkinter import ttk
# import pandas as pd
from tabulate import tabulate
from IPython.display import clear_output
import math

In [11]:
#test set
def test(num_players):
  lis=[]
  for i in range(num_players):
    lis.append("Player " + str(i+1))
  return lis

In [12]:
#setting parameters 
per_team=2
pool_groups=1
players=["Clay","David","Catherine","Nil","Ronald","Samir","Sidhant","Nehal"]
# players=test(12)

In [13]:
big_groups=len(players)/2%pool_groups
games_per_big=math.ceil(len(players)/per_team/pool_groups)*math.ceil(len(players)/per_team/pool_groups-1)/2
small_groups=pool_groups-len(players)/per_team%pool_groups
games_per_small=math.floor(len(players)/per_team/pool_groups)*math.floor(len(players)/per_team/pool_groups-1)/2
print(big_groups*games_per_big+small_groups*games_per_small)

6.0


In [27]:
#Make random teams
if len(players)%per_team!=0:
  raise Exception("Need an even number of players.")

random.shuffle(players)
teams=[]
for i in range(int(len(players)/per_team)):
  t="(" + players[per_team*i]
  if per_team==1:
    t+=")"
  for j in range(1,per_team):
    t+=" & " + players[per_team*i + j] + ")"
  teams.append(t)
  # print(t)
# teams

teams=["(Catherine & Sidhant)", "(Clay & Ronald)", "(David & Samir)", "(Nil & Nehal)"]
print(teams)

['(Catherine & Sidhant)', '(Clay & Ronald)', '(David & Samir)', '(Nil & Nehal)']


In [28]:
#Make random matchups in random groups (*if applicable)
if pool_groups*2 > len(teams):
  raise Exception("Must have at least double the amount of teams as groups.")
random.shuffle(teams)
groups=[]
for i in range(pool_groups):
    groups.append([])
for i in range(len(teams)):
    groups[i%pool_groups].append(teams[i])
matchups=[]
for g in groups:
    for m in list(combinations(g, 2)):
        matchups.append(m)
# matchups = list(combinations(teams, 2))
random.shuffle(matchups)
  
#randomize matchups
list_of_matchups=[]
for match in matchups:
  # m=random.shuffle(list(match))
  match=list(match)
  random.shuffle(match)
  mat=[match[0]]
  st=match[0]
  for j in range(1,2):
    mat.append(match[j])
    st+=" vs " + match[j]
  list_of_matchups.append(mat)
  print(st)
# list_of_matchups

(Catherine & Sidhant) vs (Clay & Ronald)
(Nil & Nehal) vs (David & Samir)
(David & Samir) vs (Clay & Ronald)
(Nil & Nehal) vs (Clay & Ronald)
(Catherine & Sidhant) vs (Nil & Nehal)
(David & Samir) vs (Catherine & Sidhant)


In [29]:
#Setting dictionary for every team.
def set_dict(teams):
    standings = dict()

    #rand is only for tie breakers
    for t in teams:
        tup={"Wins":0,"Losses":0,"Ties":0,"GF":0,"GA":0,"WF":[],"WA":[],"Rand":random.uniform(0,1)}
        standings[t]=tup

    return standings

In [30]:
#helper functions for sorting standings
def win_percentage(t, standings):
    number_of_games=(standings[t]["Wins"]+standings[t]["Losses"]+standings[t]["Ties"])
    if number_of_games==0:
        return 0
    return (standings[t]["Wins"]+0.5*standings[t]["Ties"])/number_of_games

def win_diff(t,standings):
    return standings[t]["Wins"]-standings[t]["Losses"]

def point_diff(t, standings):
    return standings[t]["GF"] - standings[t]["GA"]
    
def tied_winperc(t1,t2,standings,tied_teams=None):
    return win_percentage(t1,standings)==win_percentage(t2,standings) and win_diff(t1,standings)==win_diff(t2,standings) and \
point_diff(t1,standings) == point_diff(t2,standings) and standings[t1]["GF"] == standings[t2]["GF"]

def tied_windiff(t1,t2,standings,tied_teams):
    wins1=sum(standings[t1]["WF"].count(x) for x in tied_teams)
    losses1=sum(standings[t1]["WA"].count(x) for x in tied_teams)
    wins_against1=wins1-losses1
    
    wins2=sum(standings[t2]["WF"].count(x) for x in tied_teams)
    losses2=sum(standings[t2]["WA"].count(x) for x in tied_teams)
    wins_against2=wins2-losses2
    return wins_against1==wins_against2

#driver method for sorting.
def sort_standings(teams,standings):
    return sort(teams,standings,tied_winperc)

def sort(teams,standings,tied,tied_teams=None):
    # return sorted(teams,key=lambda t:(win_percentage(t,standings),win_diff(t,standings)),reverse=True)
    tied_count=1
    #Now we will settle tie breakers.
    for i in range(len(teams)-1):
        if tied(teams[i],teams[i+1],standings,teams):
            tied_count+=1
            # print(settle_tie(teams[i+2-tied_count:i+2],standings))
        else:
            if tied_count>1:
                teams[i+1-tied_count:i+1]=settle_tie(teams[i+1-tied_count:i+1],standings)
                # print(teams[i+2-tied_count:i+2])
            tied_count=1
    #account for last place tie.
    if tied_count>1:
        teams[-tied_count:]=settle_tie(teams[-tied_count:],standings)
        # print(teams[-tied_count:])
    return teams

#Keep commented prints for testing purposes.

def settle_tie(tied_teams,standings):
    # random.shuffle(tied_teams)
    # return tied_teams
    wins_against=dict()
    for t in tied_teams:
        wins=sum(standings[t]["WF"].count(x) for x in tied_teams)
        losses=sum(standings[t]["WA"].count(x) for x in tied_teams)
        wins_against[t]=wins-losses
    tied_teams=sorted(tied_teams,key=lambda t:wins_against[t],reverse=True)
    #convert values to list and remove duplicates. Length is 1 if it is all the same.
    if len(list(set(list(wins_against.values())))) == 1:
        tied_teams=sorted(tied_teams,key=lambda t:standings[t]["Rand"],reverse=True)
        teams_str = tied_teams[0]
        for i in range(1,len(tied_teams)):
            teams_str += ", " + tied_teams[i]
        print(teams_str + " have tied in every category. Therefore, the seeding has been determined at random.")
        # print([standings[t]["Rand"] for t in tied_teams])
        return tied_teams
    else:
        # print(wins_against)
        return sort(tied_teams,standings,tied_windiff)
    

In [31]:
scores=[(["",""]) for i in range(len(list_of_matchups))]

In [32]:
#Setting up GUI for matchups.
root = Tk()
root.title("Foosball Tournament")
root.geometry("1280x720")
# root.attributes('-toolwindow',True)

# root.configure(bg="LightBlue")
# root.iconbitmap('C:\Users\18317\Documents\Foosball\test')

my_entries=[] 

def update_scores():
    clear_output(wait=True)
    standings=set_dict(teams)
    for i in range(int(len(my_entries)/2)):
        val1= my_entries[2*i].get()
        val2= my_entries[2*i+1].get()
        scores[i]=[int(val1) if val1.isnumeric() else "",int(val2) if val2.isnumeric() else ""]
        # val1=3 if random.randint(0,1)==0 else random.randint(0,2)
        # val2=3 if val1!=3 else random.randint(0,2)
        # scores[i]=[val1,val2]
    num=0
    incomplete_count=0
    for game in scores:
        if game[0]=="" or game[1]=="":
            print(list_of_matchups[num][0] + " vs " + list_of_matchups[num][1])
            num+=1
            incomplete_count+=1
            continue
        if game[0]==game[1]:
            standings[list_of_matchups[num][0]]["Ties"]=standings[list_of_matchups[num][0]]["Ties"]+1
            standings[list_of_matchups[num][1]]["Ties"]=standings[list_of_matchups[num][1]]["Ties"]+1
            
            goals=game[0]
            standings[list_of_matchups[num][0]]["GF"]=standings[list_of_matchups[num][0]]["GF"]+goals
            standings[list_of_matchups[num][0]]["GA"]=standings[list_of_matchups[num][0]]["GA"]+goals
            standings[list_of_matchups[num][1]]["GF"]=standings[list_of_matchups[num][1]]["GF"]+goals
            standings[list_of_matchups[num][1]]["GA"]=standings[list_of_matchups[num][1]]["GA"]+goals
        else:
            ind=game.index(max(game))
            winner=list_of_matchups[num][ind]
            loser=list_of_matchups[num][(ind+1)%2]

            standings[winner]["WF"].append(loser)
            standings[winner]["Wins"]=standings[winner]["Wins"]+1
            standings[winner]["GF"]=standings[winner]["GF"]+game[ind]
            standings[winner]["GA"]=standings[winner]["GA"]+game[(ind+1)%2]

            standings[loser]["WA"].append(winner)
            standings[loser]["Losses"]=standings[loser]["Losses"]+1
            standings[loser]["GF"]=standings[loser]["GF"]+game[(ind+1)%2]
            standings[loser]["GA"]=standings[loser]["GA"]+game[ind]
        
        print(list_of_matchups[num][0] + "\t" +str(game[0]) + " - " + str(game[1]) + "\t" + list_of_matchups[num][1])
        num+=1
    col_names=["Seed","Team","GP","Wins","Losses","Ties","Win%","GF","GA","+/-"]
    table=[]
    #sort teams first by win percentage and then win/loss differential.
    sorted_teams_initial=sorted(teams,key=lambda t:(win_percentage(t,standings),win_diff(t,standings),point_diff(t,standings),standings[t]["GF"]),reverse=True)
    if incomplete_count < len(list_of_matchups)/2:
        sorted_teams=sort_standings(sorted_teams_initial.copy(),standings)
    else:
        sorted_teams=sorted_teams_initial
    seed=1
    # for t in sorted_teams:
    #     number_of_games=(standings[t]["Wins"]+standings[t]["Losses"]+standings[t]["Ties"])
    #     table.append([seed,t,number_of_games,standings[t]["Wins"],standings[t]["Losses"],standings[t]["Ties"],standings[t]["GF"],
    #                   0 if number_of_games==0 else round((standings[t]["Wins"]+0.5*standings[t]["Ties"])/number_of_games,3),
    #                   standings[t]["GF"]-standings[t]["GA"]])
    #     seed+=1
    # table=sorted(table,key=lambda x:x[0])
    # print(tabulate(table,headers=col_names, tablefmt="fancy_grid"))
    
    #test
    table=[]
    for t in sorted_teams:
        number_of_games=(standings[t]["Wins"]+standings[t]["Losses"]+standings[t]["Ties"])
        table.append([seed,t,number_of_games,standings[t]["Wins"],standings[t]["Losses"],standings[t]["Ties"],
                      0 if number_of_games==0 else round((standings[t]["Wins"]+0.5*standings[t]["Ties"])/number_of_games,3),
                      standings[t]["GF"],standings[t]["GA"],standings[t]["GF"]-standings[t]["GA"]])
        seed+=1
    table=sorted(table,key=lambda x:x[0])
    print(tabulate(table,headers=col_names, tablefmt="fancy_grid"))
    if incomplete_count>0:
        print(str(incomplete_count) + " games left to be played!")
    

main_frame = Frame(root)
main_frame.pack(fill=BOTH, expand=1)

my_canvas = Canvas(main_frame)
my_canvas.pack(side=LEFT, fill=BOTH, expand=1)

my_scrollbar = ttk.Scrollbar(main_frame, orient=VERTICAL, command=my_canvas.yview)
my_scrollbar.pack(side=RIGHT, fill=Y)

my_canvas.configure(yscrollcommand = my_scrollbar.set)
my_canvas.bind("<Configure>", lambda e: my_canvas.configure(scrollregion=my_canvas.bbox("all")))
# my_canvas.bind_all("<MouseWheel>", my_canvas.yview_scroll)

second_frame = Frame(my_canvas)

my_canvas.create_window((0,0), window=second_frame, anchor="nw")
    
for x in range(len(list_of_matchups)):
    for y in range(2):
        labelNum = Label(second_frame, text=list_of_matchups[x][y]).grid(row=2*x, column=2*y)  
        my_entry=Entry(second_frame,justify='center')
        my_entry.insert(END, str(scores[x][y]))
        my_entry.grid(row=2*x+1, column=2*y,sticky='ns',pady=(0,10))
        my_entries.append(my_entry)
    
my_button = Button(second_frame, text="Save",command=update_scores,height=1,width=10)
my_button.grid(row=1,column=y+2,padx=20)
# update_scores()

root.bind('<Return>',lambda event:update_scores())

# my_label = Label(root, text='')
# my_label.grid(row=x+2,column=0,pady=20)

root.mainloop()


(Catherine & Sidhant)	1 - 5	(Clay & Ronald)
(Nil & Nehal)	0 - 5	(David & Samir)
(David & Samir)	1 - 5	(Clay & Ronald)
(Nil & Nehal)	1 - 5	(Clay & Ronald)
(Catherine & Sidhant)	3 - 5	(Nil & Nehal)
(David & Samir)	3 - 5	(Catherine & Sidhant)
╒════════╤═══════════════════════╤══════╤════════╤══════════╤════════╤════════╤══════╤══════╤═══════╕
│   Seed │ Team                  │   GP │   Wins │   Losses │   Ties │   Win% │   GF │   GA │   +/- │
╞════════╪═══════════════════════╪══════╪════════╪══════════╪════════╪════════╪══════╪══════╪═══════╡
│      1 │ (Clay & Ronald)       │    3 │      3 │        0 │      0 │  1     │   15 │    3 │    12 │
├────────┼───────────────────────┼──────┼────────┼──────────┼────────┼────────┼──────┼──────┼───────┤
│      2 │ (David & Samir)       │    3 │      1 │        2 │      0 │  0.333 │    9 │   10 │    -1 │
├────────┼───────────────────────┼──────┼────────┼──────────┼────────┼────────┼──────┼──────┼───────┤
│      3 │ (Catherine & Sidhant) │    3 │     